In [12]:
import dlib
import cv2
import numpy as np
from scipy import ndimage

video_capture = cv2.VideoCapture(0)
glasses = cv2.imread("now_working.png",-1)
#print(glasses)

#tmp = cv2.cvtColor(glasses, cv2.COLOR_BGR2GRAY)
#_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
'''b, g, r = cv2.split(glasses)
rgba = [b,g,r, alpha]
dst = cv2.merge(rgba,4)'''
'''b_channel, g_channel, r_channel = cv2.split(glasses)

alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 50 #creating a dummy alpha channel image.

glasses = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))'''

glasses = cv2.resize(glasses,(1000,372))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#Resize an image to a certain width
def resize(img, width):
    r = float(width) / img.shape[1]
    dim = (width, int(img.shape[0] * r))
    img = cv2.resize(img, dim)
    return img

#Combine an image that has a transparency alpha channel
def blend_transparent(face_img, sunglasses_img):

    overlay_img = sunglasses_img[:,:,:3]
    #print(overlay_img)
    overlay_mask = sunglasses_img[:,:,3:]
    #print(overlay_mask)
    background_mask = 255 - overlay_mask

    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

#Find the angle between two points
def angle_between(point_1, point_2):
    angle_1 = np.arctan2(*point_1[::-1])
    angle_2 = np.arctan2(*point_2[::-1])
    return np.rad2deg((angle_1 - angle_2) % (2 * np.pi))


#Start main program
while True:

    ret, img = video_capture.read()
    img = resize(img, 600)
    img_copy = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    try:
        # detect faces
        dets = detector(gray, 1)

        #find face box bounding points
        for d in dets:

            x = d.left()
            y = d.top()
            w = d.right()
            h = d.bottom()

        dlib_rect = dlib.rectangle(x, y, w, h)

        ##############   Find facial landmarks   ##############
        detected_landmarks = predictor(gray, dlib_rect).parts()

        landmarks = np.matrix([[p.x, p.y] for p in detected_landmarks])

        for idx, point in enumerate(landmarks):
            pos = (point[0, 0], point[0, 1])
            #print(idx)
            if idx == 0:
                eye_left = pos
                print(eye_left,'left')
            elif idx == 16:
                eye_right = pos
                print(eye_right,'right')

            try:
                cv2.line(img_copy, eye_left, eye_right, color=(0, 255, 255))
                degree = np.rad2deg(np.arctan2(eye_left[0] - eye_right[0], eye_left[1] - eye_right[1]))

            except:
                pass

        ##############   Resize and rotate glasses   ##############

        #Translate facial object based on input object.

        eye_center = (eye_left[1] + eye_right[1]) / 2

        #Sunglasses translation
        glass_trans = int(.5 * (eye_center - y))  #y is forehead part

        #Funny tanslation
        #glass_trans = int(-.3 * (eye_center - y ))

        # Mask translation
        #glass_trans = int(-.8 * (eye_center - y))


        # resize glasses to width of face and blend images
        face_width = w - x

        # resize_glasses
        glasses_resize = resize(glasses, face_width)

        # Rotate glasses based on angle between eyes
        yG, xG, cG = glasses_resize.shape
        glasses_resize_rotated = ndimage.rotate(glasses_resize, (degree+90))
        glass_rec_rotated = ndimage.rotate(img[y + glass_trans:y + yG + glass_trans, x:w], (degree+90))


        #blending with rotation
        h5, w5, s5 = glass_rec_rotated.shape
        rec_resize = img_copy[y + glass_trans:y + h5 + glass_trans, x:x + w5]
        blend_glass3 = blend_transparent(rec_resize , glasses_resize_rotated)
        img_copy[y + glass_trans:y + h5 + glass_trans, x:x+w5 ] = blend_glass3
        #img_copy=cv2.cvtColor(img_copy,cv2.COLOR_RGBA2RGB)
        #b,g,r,a = cv2.split(img_copy)
        #img_copy = cv2.merge((b,g,r))
        cv2.imshow('Output', img_copy,-1)

    except:
        cv2.imshow('Output', img_copy)
        #print('fail')

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()


(242, 256) right
(455, 262) left
(243, 260) right
(455, 265) left
(242, 260) right
(453, 264) left
(241, 261) right
(452, 260) left
(241, 262) right
(452, 263) left
(241, 255) right
(455, 257) left
(237, 253) right
(452, 254) left
(235, 243) right
(443, 249) left
(232, 238) right
(434, 240) left
(225, 235) right
(411, 236) left
(218, 242) right
(395, 239) left
(216, 243) right
(396, 241) left
(217, 243) right
(402, 240) left
(217, 245) right
(402, 242) left
(222, 244) right
(410, 244) left
(223, 245) right
(410, 246) left
(219, 246) right
(408, 242) left
(212, 255) right
(396, 245) left
(213, 252) right
(397, 244) left
(224, 249) right
(407, 250) left
(219, 248) right
(406, 248) left
(221, 251) right
(409, 250) left
(220, 254) right
(407, 248) left
(220, 246) right
(408, 245) left
(221, 254) right
(409, 250) left
(221, 257) right
(410, 250) left
(224, 255) right
(409, 251) left
(222, 258) right
(411, 251) left
(224, 253) right
(411, 254) left
(222, 249) right
(411, 248) left
(224, 254)

In [4]:
import dlib
import cv2
import numpy as np
from scipy import ndimage
glasses = cv2.imread("glass.png",-1)
print(glasses.shape)
'''b_channel, g_channel, r_channel = cv2.split(glasses)

alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 50 #creating a dummy alpha channel image.

glasses = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))'''
#glasses = cv2.cvtColor(glasses, cv2.COLOR_BGR2GRAY)
glasses = cv2.resize(glasses,(1000,372),interpolation=cv2.INTER_AREA)
print(glasses.shape)

(420, 1000, 4)
(372, 1000, 4)


In [ ]:
tmp = cv2.cvtColor(glasses, cv2.COLOR_BGR2GRAY)
_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
b, g, r = cv2.split(glasses)
rgba = [b,g,r, alpha]
dst = cv2.merge(rgba,4)